In [1]:
import torch

print("Torch version :", torch.__version__)
print("CUDA dispo :", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU :", torch.cuda.get_device_name(0))

x = torch.rand(1000, 1000)
y = torch.mm(x, x)

print("CPU ok :", y.shape)

if torch.cuda.is_available():
    x = x.to("cuda")
    y = torch.mm(x, x)
    print("GPU ok :", y.shape)


Torch version : 2.5.1+cu121
CUDA dispo : True
GPU : NVIDIA GeForce RTX 3060 Laptop GPU
CPU ok : torch.Size([1000, 1000])
GPU ok : torch.Size([1000, 1000])


In [3]:
import torch
import time
import numpy as np
import csv
from pathlib import Path
import torch.nn as nn

### Si tout marche bien coté installation/fonctionnement du matos on commence


## Plan comparatif
- Opérations de reference: matmul FP32 (CPU/GPU), meme tailles N. CNN simple pour faire une demo concret
- Metriques: latence (mean/p50/p95), debit (GFLOPS), decomposition H2D/compute/D2H, energie estimee (POWER_W).
- TPU: a faire sur Colab (torch_xla ou jax faut voir ce qui marche car j'ai pas encore pu des masses faire collab car bug)
- Analyse: comparer par taille, et par energie/latence; discuter usages cibles (embarque, inference, HPC).

## TODO (samedi)
- Definir les operations de calcul de reference (matmul, CNN, ...)
- Lister le nombre/type de processeurs a tester (CPU, GPU, TPU (collab mais faire gaffe car token réduit))
- Chercher comment afficher les metriques recherchees (retour console, on stock dans un csv les valeurs puis on fait un graph de celui-ci)
- Lancer l execution et recolter les donnees (fait)
- Calculer les perfs voulues basees sur les resultats (plus définir l'utilisation optimal ? genre utiliser quoi dans quelle situation)

# Bench_matmul (CPU vs GPU)
le TPU se fait sur collab

In [ ]:
# =========================
# Configuration generale
# =========================

SIZES = { # tailles de matrices a tester
    "S": 1024,
    "M": 4096,
    "L": 8192,  # ajuster selon votre RAM/GPU
}

WARMUP = 20  # iterations de warmup
ITERS = 100
DTYPE = torch.float32

# Optionnel: forcer le nombre de threads CPU (ex: 1, 4, 8). None = auto.
CPU_THREADS = None

# Puissance moyenne en charge (W). Renseigner depuis la doc constructeur.
# i7-11800H: 45W (TDP). RTX 3060 Laptop: 80W (TGP, confirme via nvidia-smi) (ici faut changer selon votre config les gars)
POWER_W = {
    "cpu": 45,      # i7-11800H TDP 
    "gpu": 80,      # RTX 3060 Laptop TGP 
}

RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)
CSV_PATH = RESULTS_DIR / "matmul_cpu_gpu.csv"

if CPU_THREADS is not None:
    torch.set_num_threads(CPU_THREADS)

# =========================
# Outils statistiques
# =========================

def compute_stats(times_ms):
    times = np.array(times_ms)
    return {
        "mean_ms": times.mean(),
        "p50_ms": np.percentile(times, 50),
        "p95_ms": np.percentile(times, 95),
    }


def gflops(n, time_ms):
    flops = 2 * (n ** 3)
    return flops / (time_ms / 1000) / 1e9


def energy_j(power_w, time_ms):
    if power_w is None:
        return np.nan
    return power_w * (time_ms / 1000)


def device_info():
    info = {
        "torch_version": torch.__version__,
        "cpu_threads": torch.get_num_threads(),
        "cuda_available": torch.cuda.is_available(),
    }
    if torch.cuda.is_available():
        info["gpu_name"] = torch.cuda.get_device_name(0)
    return info


# =========================
# Benchmark CPU
# =========================

def bench_cpu(n):
    x = torch.rand((n, n), dtype=DTYPE)
    y = torch.rand((n, n), dtype=DTYPE)

    # Warmup
    for _ in range(WARMUP):
        torch.mm(x, y)

    times = []
    for _ in range(ITERS):
        start = time.perf_counter()
        torch.mm(x, y)
        end = time.perf_counter()
        times.append((end - start) * 1000)

    stats = compute_stats(times)
    stats["gflops"] = gflops(n, stats["mean_ms"])
    stats["h2d_ms"] = 0
    stats["compute_ms"] = stats["mean_ms"]
    stats["d2h_ms"] = 0
    stats["energy_j"] = energy_j(POWER_W["cpu"], stats["mean_ms"])
    stats["compute_energy_j"] = stats["energy_j"]

    return stats


# =========================
# Benchmark GPU
# =========================

def bench_gpu(n):
    device = "cuda"

    x_cpu = torch.rand((n, n), dtype=DTYPE, pin_memory=True)
    y_cpu = torch.rand((n, n), dtype=DTYPE, pin_memory=True)

    # Warmup complet
    for _ in range(WARMUP):
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        torch.cuda.synchronize()
        torch.mm(x, y)
        torch.cuda.synchronize()

    times = []
    h2d_list = []
    compute_list = []
    d2h_list = []

    for _ in range(ITERS):
        # H -> D
        t0 = time.perf_counter()
        x = x_cpu.to(device)
        y = y_cpu.to(device)
        torch.cuda.synchronize()
        h2d = (time.perf_counter() - t0) * 1000

        # Compute (CUDA events)
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        start.record()
        out = torch.mm(x, y)
        end.record()
        torch.cuda.synchronize()
        compute = start.elapsed_time(end)

        # D -> H
        t0 = time.perf_counter()
        out.cpu()
        torch.cuda.synchronize()
        d2h = (time.perf_counter() - t0) * 1000

        total = h2d + compute + d2h
        times.append(total)
        h2d_list.append(h2d)
        compute_list.append(compute)
        d2h_list.append(d2h)

    stats = compute_stats(times)
    stats["gflops"] = gflops(n, stats["mean_ms"])
    stats["h2d_ms"] = float(np.mean(h2d_list))
    stats["compute_ms"] = float(np.mean(compute_list))
    stats["d2h_ms"] = float(np.mean(d2h_list))
    stats["energy_j"] = energy_j(POWER_W["gpu"], stats["mean_ms"])
    stats["compute_energy_j"] = energy_j(POWER_W["gpu"], stats["compute_ms"])

    return stats


# =========================
# Sauvegarde CSV
# =========================

def save_row(device, size_tag, n, stats):
    file_exists = CSV_PATH.exists()
    with open(CSV_PATH, "a", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow([
                "device", "size", "N",
                "mean_ms", "p50_ms", "p95_ms",
                "gflops",
                "h2d_ms", "compute_ms", "d2h_ms",
                "power_w", "energy_j", "compute_energy_j",
            ])

        power_w = POWER_W[device]
        writer.writerow([
            device, size_tag, n,
            stats["mean_ms"], stats["p50_ms"], stats["p95_ms"],
            stats["gflops"],
            stats["h2d_ms"], stats["compute_ms"], stats["d2h_ms"],
            power_w, stats["energy_j"], stats["compute_energy_j"],
        ])


# =========================
# Execution principale
# =========================

def main():
    info = device_info()
    print("Infos:", info)
    print("Debut des tests de multiplication matricielle\n")
    print("\nOn collecte: taille N, mean_ms, p50_ms, p95_ms, gflops.")
    print("GPU: h2d_ms, compute_ms, d2h_ms pour decomposer le temps.")
    print("Energie: power_w * temps (renseigner POWER_W).\n")

    for size_tag, n in SIZES.items():
        print(f"\n=== Taille {size_tag} (N={n}) ===")

        # CPU
        cpu_stats = bench_cpu(n)
        print("CPU:", cpu_stats)
        save_row("cpu", size_tag, n, cpu_stats)

        # GPU
        if torch.cuda.is_available():
            gpu_stats = bench_gpu(n)
            print("GPU:", gpu_stats)
            save_row("gpu", size_tag, n, gpu_stats)


if __name__ == "__main__":
    main()

Infos: {'torch_version': '2.5.1+cu121', 'cpu_threads': 8, 'cuda_available': True, 'gpu_name': 'NVIDIA GeForce RTX 3060 Laptop GPU'}
Debut des tests de multiplication matricielle


On collecte: taille N, mean_ms, p50_ms, p95_ms, gflops.
GPU: h2d_ms, compute_ms, d2h_ms pour decomposer le temps.
Energie: power_w * temps (renseigner POWER_W).


=== Taille S (N=1024) ===
CPU: {'mean_ms': 10.805466000019806, 'p50_ms': 11.014000003342517, 'p95_ms': 11.836015001972555, 'gflops': 198.7404937460415, 'h2d_ms': 0, 'compute_ms': 10.805466000019806, 'd2h_ms': 0, 'energy_j': 0.4862459700008912, 'compute_energy_j': 0.4862459700008912}
GPU: {'mean_ms': 7.073579716985114, 'p50_ms': 6.625951990747126, 'p95_ms': 9.120495578640838, 'gflops': 303.59220280552603, 'h2d_ms': 3.1194089998462005, 'compute_ms': 2.340758717060089, 'd2h_ms': 1.6134120000788243, 'energy_j': 0.5658863773588091, 'compute_energy_j': 0.18726069736480716}

=== Taille M (N=4096) ===
CPU: {'mean_ms': 604.3914229998336, 'p50_ms': 602.600649

Les résultats du test MatMul montrent que le GPU (RTX 3060) n’apporte qu’un gain limité pour les petites matrices (S) en raison du coût des transferts mémoire et de la sous-utilisation de celui-ci.


En revanche, pour des tailles moyennes et grandes (M et L), le GPU atteint un débit d’environ 3,2 TFLOPS, soit un facteur d’accélération proche de ×6 par rapport au CPU.


Résultats principaux

Petites matrices (1024) :
le GPU est seulement légèrement plus rapide, car les transferts mémoire dominent.
--> Le CPU reste pertinent pour de petites charges.

Matrices moyennes (4096) :
le GPU devient ≈ 4× plus rapide.
--> Le parallélisme GPU commence à être pleinement exploité.

Grandes matrices (8192) :
le GPU atteint --> 10× la performance du CPU tout en consommant beaucoup moins d’énergie,
même si ~40 % du temps reste lié aux transferts mémoire.


### Points clés à retenir

Le CPU est adapté aux petites tâches ou aux traitements séquentiels.

Le GPU devient dominant lorsque l’intensité de calcul augmente.

Les transferts mémoire constituent un goulot d’étranglement réel.

Les workloads d’IA massifs privilégient donc les GPU ou accélérateurs spécialisés.

# Bench_CNN (CPU vs GPU)
le TPU se fait sur collab

In [ ]:
# =========================
# Configuration
# =========================

BATCH_SIZE = 32
IMAGE_SIZE = 64
WARMUP = 20
ITERS = 100
DTYPE = torch.float32

POWER_W = {
    "cpu": 45,   # comme d'habitude, adapter à votre CPU, GPU
    "gpu": 80,   # i7-11800H TDP # RTX 3060 Laptop TGP 
}

RESULTS_DIR = Path("results")
RESULTS_DIR.mkdir(exist_ok=True)
CSV_PATH = RESULTS_DIR / "cnn_cpu_gpu.csv"


# =========================
# Modèle CNN simple
# =========================

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * (IMAGE_SIZE // 4) * (IMAGE_SIZE // 4), 128),
            nn.ReLU(),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# =========================
# Outils statistiques
# =========================

def compute_stats(times_ms):
    t = np.array(times_ms)
    return {
        "mean_ms": float(t.mean()),
        "p50_ms": float(np.percentile(t, 50)),
        "p95_ms": float(np.percentile(t, 95)),
    }


def energy_j(power_w, time_ms):
    return power_w * (time_ms / 1000)


# =========================
# Benchmark CPU
# =========================

def bench_cpu():
    device = "cpu"

    model = SimpleCNN().to(device)
    model.eval()

    x = torch.rand((BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE), dtype=DTYPE)

    # Warmup
    with torch.no_grad():
        for _ in range(WARMUP):
            model(x)

    times = []

    with torch.no_grad():
        for _ in range(ITERS):
            t0 = time.perf_counter()
            model(x)
            t1 = time.perf_counter()
            times.append((t1 - t0) * 1000)

    stats = compute_stats(times)
    stats["energy_j"] = energy_j(POWER_W["cpu"], stats["mean_ms"])
    return stats


# =========================
# Benchmark GPU
# =========================

def bench_gpu():
    device = "cuda"

    model = SimpleCNN().to(device)
    model.eval()

    x_cpu = torch.rand((BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE), dtype=DTYPE, pin_memory=True)

    # Warmup
    with torch.no_grad():
        for _ in range(WARMUP):
            x = x_cpu.to(device, non_blocking=True)
            torch.cuda.synchronize()
            model(x)
            torch.cuda.synchronize()

    times = []
    h2d_list, compute_list = [], []

    with torch.no_grad():
        for _ in range(ITERS):

            # H2D
            t0 = time.perf_counter()
            x = x_cpu.to(device, non_blocking=True)
            torch.cuda.synchronize()
            h2d = (time.perf_counter() - t0) * 1000

            # Compute
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)

            start.record()
            model(x)
            end.record()

            torch.cuda.synchronize()
            compute = start.elapsed_time(end)

            times.append(h2d + compute)
            h2d_list.append(h2d)
            compute_list.append(compute)

    stats = compute_stats(times)
    stats["h2d_ms"] = float(np.mean(h2d_list))
    stats["compute_ms"] = float(np.mean(compute_list))
    stats["energy_j"] = energy_j(POWER_W["gpu"], stats["mean_ms"])
    return stats


# =========================
# Sauvegarde CSV
# =========================

def save_row(device, stats):
    file_exists = CSV_PATH.exists()

    with open(CSV_PATH, "a", newline="") as f:
        writer = csv.writer(f)

        if not file_exists:
            writer.writerow([
                "device",
                "mean_ms", "p50_ms", "p95_ms",
                "h2d_ms", "compute_ms",
                "energy_j",
            ])

        writer.writerow([
            device,
            stats["mean_ms"], stats["p50_ms"], stats["p95_ms"],
            stats.get("h2d_ms", 0),
            stats.get("compute_ms", stats["mean_ms"]),
            stats["energy_j"],
        ])


# =========================
# Exécution
# =========================

def main():
    print("=== Benchmark CNN (CPU vs GPU) ===\n")

    cpu_stats = bench_cpu()
    print("CPU:", cpu_stats)
    save_row("cpu", cpu_stats)

    if torch.cuda.is_available():
        gpu_stats = bench_gpu()
        print("GPU:", gpu_stats)
        save_row("gpu", gpu_stats)


if __name__ == "__main__":
    main()


=== Benchmark CNN simple (CPU vs GPU) ===

CPU: {'mean_ms': 6.871380000084173, 'p50_ms': 6.945799996174173, 'p95_ms': 7.6069350019679405, 'energy_j': 0.3092121000037878}
GPU: {'mean_ms': 0.7174640369974077, 'p50_ms': 0.6905280048085842, 'p95_ms': 0.890744613468996, 'h2d_ms': 0.16592100029811263, 'compute_ms': 0.551543036699295, 'energy_j': 0.057397122959792615}


Bon, même si le CNN utilisé ici reste volontairement simple, on voit déjà très clairement que le GPU devient beaucoup plus intéressant dès que la charge de calcul augmente.
On passe d’environ 6,9 ms par inférence sur CPU à 0,7 ms sur GPU, soit presque 10× plus rapide, ce qui confirme que les réseaux convolutionnels exploitent très bien le parallélisme massif des GPU

le CNN testé prend en entrée des images RGB de 64×64 avec un batch size de 32.
L’architecture est composée de deux blocs convolutionnels (convolution + ReLU + max-pooling) 
Le benchmark est réalisé avec un warm-up comme pour le matmul pour stabiliser les performances puis plusieurs itérations pour mesurer une latence moyenne fiable,
Côté GPU, on sépare aussi le temps de transfert mémoire et le temps de calcul réel.

On voit pour commencer que le temps de transfert vers le GPU existe toujours, mais qu’il reste relativement faible par rapport au gain énorme en temps de calcul.
On observe aussi que le GPU consomme moins d’énergie par inférence que le CPU, simplement parce qu’il termine le calcul beaucoup plus vite.